# Notebook for Creating Figures from OpenFOAM Solutions

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Define Helper Files

In [2]:
def camera_settings(VIEW='pylon'):
    """ Define settings for viewing the figures """
    if VIEW == 'pylon':
        camera = pv.Camera()
        camera.position = (15.9143, 3.83627, 28.3582)
        camera.focal_point = (15.9143, 3.83627, 13.75)
        camera.view_angle = 30
    elif VIEW == 'wing':
        camera = pv.Camera()
        camera.position = (6.754050981213595, 2.974053164557754, 13.524279730513474)
        camera.focal_point = (14.999999523162842, 2.974053164557754, 13.524279730513474)
        camera.view_angle = 30
    else:
        camera = pv.Camera()
        camera.position = (14.200957354542979, 0.9359974688819834, 55.47461841512844)
        camera.focal_point = (14.200957354542979, 0.9359974688819834, 1.702326812846877e-08)
        camera.view_angle = 30
    
    return camera

In [3]:
def slice_center(mesh, NORMAL='z', ORIGIN=[0, 0, 0], translate=False, CONTOUR=False):
    """Slice mesh through center in normal direction, move to zero normal."""
    slice_mesh = mesh.slice(normal=NORMAL, origin=ORIGIN, contour=CONTOUR)
    if translate is not False:
        if NORMAL == 'z':
            slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
        elif NORMAL == 'x':
            slice_mesh.translate((-slice.mesh.center[0], 0, 0), inplace=True)
        elif NORMAL == 'y':
            slice_mesh.translate((0, -slice.mesh.center[1], 0), inplace=True)
        else:
            print('ERROR: Normal not found - ', NORMAL)
            slice_mesh = None
    return slice_mesh

In [4]:
def read_mesh(nfile):
    """ Read the OpenFOAM solution, needs the file generated by paraFoam (e.g. *.OpenFOAM) """
    # Read the reference file created by openFOAM `parafoam`
    reader = pv.POpenFOAMReader(nfile)
    # set the active time 
    reader.set_active_time_value(reader.time_values[-1])
    reader.cell_to_point_creation = True
    # Define the mesh
    mesh = reader.read()
    # Define the internal mesh and boundaries
    internal = mesh["internalMesh"]
    boundaries = mesh["boundary"]
    
    return internal, boundaries

In [5]:
def sample_line(mesh, pointA, pointB, field=None):
    """ Sample fields across a line, marked by two points, within the mesh """
    # Sample across the mesh designated by inputed points
    line = mesh.sample_over_line(pointA,
                                 pointB,
                                 resolution=100,
                                 tolerance=0.2,
                                 )
    # Define the distance of this line
    distance = line['Distance']
    
    # Check to see if a desired field to extract
    if field is not None:
        subset = line.get_array(field)
    else:
        subset = None
    
    if field is not None:
        return distance, subset
    else:
        return distance, line

In [6]:
def plot_field(ndir, 
               nfile,
               FIELD='U', 
               RANGE=[95, 145], 
               CONTOUR=True, 
               ORIGIN=[0, 0, 0], 
               NORMAL='z', 
               CMAP='bwr',
               SAVE=False,
               VIEW='pylon',
               LINE=False):
    """ 
    Create 2-D Figure of the OpenFOAM internal mesh for selected field. 
    Normal decides which plane to exclude from display.
    Origin moves where the slice is preformed.
    """
    # add extension to the input directory file to reference *.OpenFOAM files
    solution = ndir + nfile['filename'] + nfile['filename'][:-1] + '.OpenFoam'
    
    # Define label for input field
    LABEL = {"U" : "Velocity [m/s]",
             "p" : "Pressure [Pa]", 
             "T" : "Temperature [C]"
            }
    
    # Read the Mesh
    internal, boundaries = read_mesh(solution)
    
    # Slice the Mesh
    if VIEW == 'pylon':
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=nfile['pylon'])
    else:
        # fuselage
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=ORIGIN)
    
    # Contour the Mesh
    contours = p3_slice.contour(scalars=FIELD, isosurfaces=15, rng=RANGE)
    
    # Set a custom position and size
    sargs = dict(height=0.10, 
                 vertical=False, 
                 position_x=0.2, 
                 position_y=0.05, 
                 fmt="%.0f", 
                 color='black', 
                 n_labels=6, 
                 title=LABEL[FIELD])
    
    # Create the plotter
    pt = pv.Plotter()
    # Add the slice mesh
    pt.add_mesh(p3_slice, 
                scalars=FIELD, 
                preference='point', 
                cmap=CMAP, 
                clim=RANGE,
                scalar_bar_args=sargs)
    
    # Add the contours
    pt.add_mesh(contours, 
                color='white', 
                preference='cell', 
                line_width=1)
    
    # Add a ruler
    if VIEW == 'pylon':
        pt.add_ruler(pointa=[10, -0.35, 0.0],
                     pointb=[20, -0.35, 0.0],
                     title='Distance [m]',
                     tick_color='black',
                     label_color='black'
                    )
    elif VIEW == 'wing':
        pt.add_ruler(pointa=[15, 2.5, 11.5],
             pointb=[15, 2.5, 15.5],
             title='Distance [m]',
             tick_color='black',
             label_color='black'
            )
    else:
        # fuselage view
        pt.add_ruler(pointa=[-5, -2.5, 0.0],
                     pointb=[35, -2.5, 0.0],
                     title='Distance [m]',
                     tick_color='black',
                     label_color='black'
                    )
   
    # Add a title
    pt.add_title(nfile['filename'][:-1], 
                 font='courier', 
                 color='k', 
                 font_size=10
                )
    
    # Define the camera view
    pt.camera = camera_settings(VIEW=VIEW)
    
    # Save the figure
    if SAVE is True:
        if VIEW == 'pylon':
            # Define a figure title
            nout = '/Users/jrobrien/dev/dissertation/figures/' + nfile['filename'][:-1] + '_PYLON_' + str(FIELD) + '.svg'
            # Save the figure
            pt.save_graphic(nout, raster=True)
        elif VIEW == 'wing':
            # Define a figure title
            nout = '/Users/jrobrien/dev/dissertation/figures/' + nfile['filename'][:-1] + '_WING_' + str(FIELD) + '.svg'
            # Save the figure
            pt.save_graphic(nout, raster=True)
        else:
            # fuselage
            # Define a figure title
            nout = '/Users/jrobrien/dev/dissertation/figures/' + nfile['filename'][:-1] + '_FUSELAGE_' + str(FIELD) + '.svg'
            # Save the figure
            pt.save_graphic(nout, raster=True)
        
    if LINE is True:
        # Make two points to construct the line between
        A = [13.5, nfile['pylon'][1], nfile['pylon'][2]]
        B = [14.49, nfile['pylon'][1], nfile['pylon'][2]]

        # Preview how this line intersects this mesh
        line = pv.Line(A, B)
        pt.add_mesh(line, color="white", line_width=10)
        
    # Display the figure
    pt.show()
    
    # Free up some memory
    del internal, p3_slice, contours, boundaries, pt

In [22]:
def plot_line(ndir,
              nfiles,
              FIELD='U',
              reference=False,
              NORMAL='z',
              ORIGIN=[0,0,0],
              SAVE=True,
              **kwargs):
    """
    Create line plot for selected pylon against all angles of attack and reference solutions
    
    Parameters
    ----------
    ndir = str
        Directory holding these solutions
    nfiles = dict
        Dictionary holding pylon configuration directory names, CDP location, etc
    FIELD = str
        Scalar to display
    reference = boolean, Default=False
        If True, compare solutions against No-Pylon solutions
        
    Returns
    -------
    fig - figure plotting sampled line
    """
     # Define label for input field
    LABEL = {"U" : "Velocity [m/s]",
             "p" : "Pressure [Pa]", 
             "T" : "Temperature [C]"
            }
    
    # define a blank dictionary to hold the sample over line
    solutions = {}

    for pylon in nfiles:
        # add extension to the input directory file to reference *.OpenFOAM files
        directory = ndir + pylon['filename'] + pylon['filename'][:-1] + '.OpenFoam'
    
        # Read the Mesh
        internal, boundaries = read_mesh(directory)
    
        # Slice the Mesh
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=pylon['pylon'])
            
        # Sample over line
        disx, field = sample_line(p3_slice, pylon['start'], pylon['end'])
        
        solutions[pylon['tag']] = {'distance' : disx,
                                   'scalar' : field
                                  }
        # clear up some memory
        del internal, boundaries, p3_slice, disx, field
        
    # if reference is set, also extract the no pylon solutions
    # COMING SOON!
    
    # Display
    fig, axarr = plt.subplots(1, 1, figsize=[12, 8])
    for case in solutions:
        if FIELD == 'U':
            axarr.plot(solutions[case]['distance']-.1, 
                       solutions[case]['scalar'].point_data['U'][:, 0], 
                       label=case + '_Ux')
            axarr.plot(solutions[case]['distance']-.1, 
                       solutions[case]['scalar'].point_data['U'][:, 1],
                       linestyle='--',
                       label=case + '_Uy')
            axarr.plot(solutions[case]['distance']-.1, 
                       solutions[case]['scalar'].point_data['U'][:, 2], 
                       linestyle='-.',
                       label=case + '_Uz')
        else:
            axarr.plot(solutions[case]['distance']-.1,
                       solutions[case]['scalar'],
                       label=case + FIELD)
    axarr.axvline(x=-0.038, ymin=-30, ymax=200)
    # Define some figure comments
    if FIELD == 'U':
        axarr.set_ylim([-35, 160])
    axarr.set_xlabel('Distance from Cloud Droplet Probe Strut [m]')
    axarr.set_ylabel('Velocity [m/s]')
    axarr.grid(True)
    plt.legend(loc='center left')
    
    if SAVE is True:
        tmpx = [x for x in solutions]
        # Define a figure title
        nout = '/Users/jrobrien/dev/dissertation/figures/' + 'line_sample_' + tmpx[0] + '_' + str(FIELD) + '.svg'
        # Save the figure
        plt.savefig(nout)

In [38]:
def sample_volume(ndir,
                  nfiles,
                  FIELD='U',
                  reference=False,
                  NORMAL='z',
                  ORIGIN=[0,0,0],
                  **kwargs
                 ):
    """ Grab velocity values within the CDP sample volume location """
    # define a blank dictionary to hold the sample over line
    solutions = {}

    for pylon in nfiles:
        # add extension to the input directory file to reference *.OpenFOAM files
        directory = ndir + pylon['filename'] + pylon['filename'][:-1] + '.OpenFoam'
    
        # Read the Mesh
        internal, boundaries = read_mesh(directory)
    
        # Slice the Mesh
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=pylon['pylon'])
            
        # Sample over line
        disx, field = sample_line(p3_slice, pylon['start'], pylon['pylon'])
        
        solutions[pylon['tag']] = {'distance' : disx,
                                   'scalar' : field
                                  }
        # Display the sample volume values
        print(pylon['tag'], field.point_data['U'][-1])
        
        # clear up some memory
        del internal, boundaries, p3_slice, disx, field
        
    return solutions
    

## Define the Directory, Case Structure, Fields and Pylon Locations

In [54]:
# Define the directory hosting the solution directories
solution_dir = "/Users/jrobrien/Dissertation/data/solutions/"
# Define available solution directories
case_120 = {'navy_120_aoa0' : {'filename' : 'NASA_navyPylon_v2_tas120_aoa0_900T33/',
                               'start' : [14.39, 3.05, 13.75],
                               'end' : [14.49, 3.05, 13.75],
                               'pylon' : [14.452, 3.05, 13.75],
                               'tag' : 'Navy120_AoA0'
                              },
            'extend_120_aoa0' : {'filename' : 'NASA_extendedPylon_v2_tas120_aoa0_900T33/',
                                 'start' : [13.82, 2.725, 13.7],
                                 'end' : [13.92, 2.725, 13.7],
                                 'pylon' : [13.882, 2.725, 13.7],
                                 'tag' : 'Extend120_AoA0',
                                },
            'no_120_aoa0' : {'filename' : 'NASA_noPylons_v2_tas120_aoa0_900T33/',
                                 'start' : [13.82, 2.725, 13.7],
                                 'end' : [13.92, 2.725, 13.7],
                                 'pylon' : [13.882, 2.725, 13.7],
                             'tag' : 'Null120_AoA0',
                            },
            'navy_120_aoaPos4' : {'filename' : 'NASA_navyPylon_v3_tas120_aoaPos4_900T33/',
                                  'start' : [14.39, 3.05, 13.75],
                                  'end' : [14.49, 3.05, 13.75],
                                  'pylon' : [14.452, 3.05, 13.75],
                                  'tag' : 'Navy120_AoA+4',
                                 },
            'extend_120_aoaPos4' : {'filename' : 'NASA_extendedPylon_v3_tas120_aoaPos4_900T33/',
                                    'start' : [13.82, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7],
                                    'pylon' : [13.882, 2.725, 13.7],
                                    'tag' : 'Extend120_AoA+4',
                                   },
            'no_120_aoaPos4' : {'filename' : 'NASA_noPylons_v3_tas120_aoaPos4_900T33/',
                                'pylon' : [0, 2.75, 13.7],
                                'tag' : 'Null120_AoA+4',
                               },
            'navy_120_aoaNeg4' : {'filename' : 'NASA_navyPylon_v3_tas120_aoaNeg4_900T33/',
                                  'start' : [14.39, 3.05, 13.75],
                                  'end' : [14.49, 3.05, 13.75],
                                  'pylon' : [14.452, 3.05, 13.75],
                                  'tag' : 'Navy120_AoA-4',
                                 },
            'extend_120_aoaNeg4' : {'filename' : 'NASA_extendedPylon_v3_tas120_aoaNeg4_900T33/',
                                    'start' : [13.82, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7],
                                    'pylon' : [13.882, 2.725, 13.7],
                                    'tag' : 'Extend120_AoA-4',
                                   },
            'no_120_aoaNeg4' : {'filename' : 'NASA_noPylons_v3_tas120_aoaNeg4_900T33/',
                                'pylon' : [0, 2.75, 13.7],
                                'tag' : 'Null120_AoA-4'
                               }
           }
# Define available solution directories
case_140 = {'navy_140_aoa0' : {'filename' : 'NASA_navyPylon_v3_tas140_aoa0_800T20/',
                               'start' : [14.39, 3.05, 13.75],
                               'end' : [14.49, 3.05, 13.75],
                               'pylon' : [14.452, 3.05, 13.75],
                               'tag' : 'Navy140_AoA0'
                              },
            'extend_140_aoa0' : {'filename' : 'NASA_extendedPylon_v3_tas140_aoa0_800T20/',
                                 'start' : [13.82, 2.725, 13.7],
                                 'end' : [13.92, 2.725, 13.7],
                                 'pylon' : [13.882, 2.725, 13.7],
                                 'tag' : 'Extend140_AoA0',
                                },
            'no_140_aoa0' : {'filename' : 'NASA_noPylons_v3_tas140_aoa0_800T20/',
                             'pylon' : [0, 2.75, 13.7],
                             'tag' : 'Null140_AoA0',
                            },
            'navy_140_aoaPos4' : {'filename' : 'NASA_navyPylon_v3_tas140_aoaPos4_800T20/',
                                  'start' : [14.39, 3.05, 13.75],
                                  'end' : [14.49, 3.05, 13.75],
                                  'pylon' : [14.452, 3.05, 13.75],
                                  'tag' : 'Navy140_AoA+4'
                                 },
            'extend_140_aoaPos4' : {'filename' : 'NASA_extendedPylon_v3_tas140_aoaPos4_800T20/',
                                    'start' : [13.82, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7],
                                    'pylon' : [13.882, 2.725, 13.7],
                                    'tag' : 'Extend140_AoA+4',
                                   },
            'no_140_aoaPos4' : {'filename' : 'NASA_noPylons_v3_tas140_aoaPos4_800T20/',
                                'pylon' : [0, 2.75, 13.7],
                                'tag' : 'Null140_AoA+4',
                               },
            'navy_140_aoaNeg4' : {'filename' : 'NASA_navyPylon_v3_tas140_aoaNeg4_800T20/',
                                  'start' : [14.39, 3.05, 13.75],
                                  'end' : [14.49, 3.05, 13.75],
                                  'pylon' : [14.452, 3.05, 13.75],
                                  'tag' : 'Navy140_AoA-4',
                                 },
            'extend_140_aoaNeg4' : {'filename' : 'NASA_extendedPylon_v3_tas140_aoaNeg4_800T20/',
                                    'start' : [13.82, 2.725, 13.7],
                                    'end' : [13.92, 2.725, 13.7],
                                    'pylon' : [13.882, 2.725, 13.7],
                                    'tag' : 'Extend140_AoA-4',
                                   },
            'no_140_aoaNeg4' : {'filename' : 'NASA_noPylons_v3_tas140_aoaNeg4_800T20/',
                                'pylon' : [0, 2.75, 13.7],
                                'tag' : 'Null140_AoA-4',
                               }
           }

In [10]:
scalar_field_120 = {'U' : {'range' : [95, 145]},
                    'T' : {'range' : [295, 318]},
                    'p' : {'range' : [89000, 91000]}
                   }
scalar_field_140 = {'U' : {'range' : [115, 165]},
                    'T' : {'range' : [281, 305]},
                    'p' : {'range' : [79000, 81000]}
                   }

## Process the OpenFOAM Solutions

### A) Process the Solutions for TAS=120 - Along Pylon

In [56]:
# Process the selected Scalar fields
for case in case_120:
    if case.startswith('no_') is False:
        for scalar in scalar_field_120:
                print(case_120[case])
                plot_field(solution_dir, 
                           case_120[case], 
                           SAVE=True, 
                           FIELD=scalar, 
                           RANGE=scalar_field_120[scalar]['range'], 
                           VIEW='pylon',
                           LINE=True)

{'filename': 'NASA_navyPylon_v2_tas120_aoa0_900T33/', 'start': [14.39, 3.05, 13.75], 'end': [14.49, 3.05, 13.75], 'pylon': [14.452, 3.05, 13.75], 'tag': 'Navy120_AoA0'}


2023-06-08 23:48:03.435 (16973.874s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Widget(value="<iframe src='http://localhost:55013/index.html?ui=P_0x13ffdac40_0&reconnect=auto' style='width: …

{'filename': 'NASA_navyPylon_v2_tas120_aoa0_900T33/', 'start': [14.39, 3.05, 13.75], 'end': [14.49, 3.05, 13.75], 'pylon': [14.452, 3.05, 13.75], 'tag': 'Navy120_AoA0'}


2023-06-08 23:48:16.270 (16986.709s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


...

Widget(value="<iframe src='http://localhost:55013/index.html?ui=P_0x13fee3340_15&reconnect=auto' style='width:…

{'filename': 'NASA_extendedPylon_v3_tas120_aoaNeg4_900T33/', 'start': [13.82, 2.725, 13.7], 'end': [13.92, 2.725, 13.7], 'pylon': [13.882, 2.725, 13.7], 'tag': 'Extend120_AoA-4'}


Widget(value="<iframe src='http://localhost:55013/index.html?ui=P_0x13fee3fa0_16&reconnect=auto' style='width:…

{'filename': 'NASA_extendedPylon_v3_tas120_aoaNeg4_900T33/', 'start': [13.82, 2.725, 13.7], 'end': [13.92, 2.725, 13.7], 'pylon': [13.882, 2.725, 13.7], 'tag': 'Extend120_AoA-4'}


Widget(value="<iframe src='http://localhost:55013/index.html?ui=P_0x13c6dbe50_17&reconnect=auto' style='width:…

### B) Process the Solutions for TAS=120 - Along Wing

In [ ]:
# Process the selected Scalar fields
for case in case_120:
    if case.startswith('no_') is False:
            for scalar in scalar_field_120:
                if case.startswith('navy_') is True:
                    plot_field(solution_dir, 
                               case_120[case], 
                               SAVE=True, 
                               FIELD=scalar,
                               NORMAL='x',
                               ORIGIN=[15, 0, 0],
                               RANGE=scalar_field_120[scalar]['range'], 
                               VIEW='wing',
                               LINE=False)
                if case.startswith('extend_') is True:
                    plot_field(solution_dir, 
                               case_120[case], 
                               SAVE=True, 
                               FIELD=scalar,
                               NORMAL='x',
                               ORIGIN=[14.5, 0, 0],
                               RANGE=scalar_field_120[scalar]['range'], 
                               VIEW='wing',
                               LINE=False)

### C) Process the Solutions for TAS=120 - Along Fuselage

In [ ]:
# Process the selected Scalar fields
for case in case_120:
    if case.startswith('no_') is False:
            for scalar in scalar_field_120:
                    plot_field(solution_dir, 
                               case_120[case], 
                               SAVE=True, 
                               FIELD=scalar,
                               RANGE=scalar_field_120[scalar]['range'], 
                               VIEW='fuselage',
                               LINE=False)

### D) Process the Solutions for TAS=140 - Pylon

In [ ]:
# Process the selected Scalar fields
for case in case_140:
    if case.startswith('no_') is False:
        for scalar in scalar_field_140:
                print(case_140[case])
                plot_field(solution_dir, 
                           case_140[case], 
                           SAVE=True, 
                           FIELD=scalar, 
                           RANGE=scalar_field_140[scalar]['range'], 
                           VIEW='pylon',
                           LINE=False)

### E) Process the Solutions for TAS=140 - Along wing

In [ ]:
# Process the selected Scalar fields
for case in case_140:
    if case.startswith('no_') is False:
            for scalar in scalar_field_140:
                plot_field(solution_dir, 
                           case_140[case], 
                           SAVE=True, 
                           FIELD=scalar,
                           NORMAL='x',
                           ORIGIN=[15, 0, 0],
                           RANGE=scalar_field_140[scalar]['range'], 
                           VIEW='wing',
                           LINE=False)

### F) Process the Solutions for TAS=140 - Along fuselage

In [ ]:
# Process the selected Scalar fields
for case in case_140:
    if case.startswith('no_') is False:
            for scalar in scalar_field_140:
                plot_field(solution_dir, 
                           case_140[case], 
                           SAVE=True, 
                           FIELD=scalar, 
                           RANGE=scalar_field_140[scalar]['range'], 
                           VIEW='fuselage',
                           LINE=False)

### G) Sample Over Line - Navy Pylon

In [39]:
extend_120_line= [case_120['extend_120_aoa0'], case_120['extend_120_aoaPos4'], case_120['extend_120_aoaNeg4']]
extend_140_line= [case_140['extend_140_aoa0'], case_140['extend_140_aoaPos4'], case_140['extend_140_aoaNeg4']]

In [ ]:
# Process the selected Scalar fields over the line
plot_line(solution_dir, 
          extend_120_line, 
          FIELD='U',
          SAVE=True)

In [ ]:
# Process the selected Scalar fields over the line
plot_line(solution_dir, 
          extend_140_line, 
          FIELD='U',
          SAVE=True)

In [40]:
navy_120_line = [case_120['navy_120_aoa0'], case_120['navy_120_aoaPos4'], case_120['navy_120_aoaNeg4']]
navy_140_line = [case_140['navy_140_aoa0'], case_140['navy_140_aoaPos4'], case_140['navy_140_aoaNeg4']]

In [ ]:
# Process the selected Scalar fields over the line
plot_line(solution_dir, 
          navy_120_line, 
          FIELD='U',
          SAVE=True)

In [ ]:
# Process the selected Scalar fields over the line
plot_line(solution_dir, 
          navy_140_line, 
          FIELD='U',
          SAVE=True)

### H) Value within CDP Sample Volume

In [41]:
cdp_navy_120 = sample_volume(solution_dir,
                             navy_120_line,
                             FIELD='U'
                            )

Navy120_AoA0 [109.07605  -10.584614   6.992657]


2023-06-08 19:21:12.458 ( 962.860s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-08 19:21:23.863 ( 974.265s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Navy120_AoA+4 [102.987175   -1.4845028  10.082008 ]
Navy120_AoA-4 [115.89747  -19.956003   4.498146]


2023-06-08 19:21:35.325 ( 985.727s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


In [44]:
cdp_navy_140 = sample_volume(solution_dir,
                             navy_140_line,
                             FIELD='U'
                            )

Navy140_AoA0 [126.96852  -12.557798   8.403991]


2023-06-08 19:30:44.518 (1534.975s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Navy140_AoA+4 [119.18748    -1.9579902  11.982219 ]
Navy140_AoA-4 [134.44511   -23.40407     5.4121456]


2023-06-08 19:30:55.921 (1546.378s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-08 19:31:07.399 (1557.856s) [           33EEA]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


In [45]:
cdp_extend_120 = sample_volume(solution_dir,
                             extend_120_line,
                             FIELD='U'
                            )

Extend120_AoA0 [ 71.647934 -16.372137   3.07285 ]
Extend120_AoA+4 [68.46866   -5.6719894  4.52588  ]
Extend120_AoA-4 [ 71.50642  -25.49675    2.617618]


In [46]:
cdp_extend_140 = sample_volume(solution_dir,
                             extend_140_line,
                             FIELD='U'
                            )

Extend140_AoA0 [ 82.543625  -19.152905    3.6805859]
Extend140_AoA+4 [78.87019   -6.677492   5.2871346]
Extend140_AoA-4 [ 82.25195   -29.884253    3.1250114]


In [55]:
no_line = [case_120['no_120_aoa0']]
no_120 = sample_volume(solution_dir,
                       no_line,
                       FIELD='U'
                       )

Null120_AoA0 [118.49963    -1.5518469   3.0030942]
